In [1]:
import pandas as pd
from fuzzywuzzy import process
from collections import Counter
import re


C:\Users\mofei\AppData\Roaming\Python\Python39\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
df = pd.read_csv('Data/merged_df_1021_p2_p4filter.csv', sep='|')

C:\Users\mofei\AppData\Local\Temp\ipykernel_13632\769174394.py:1: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Data/merged_df_1021_p2_p4filter.csv', sep='|')


In [3]:
dt = pd.read_csv('Data/disease types.csv', header=None, names=["disease_type"]).replace({'"': ''}, regex=True)
con = list(df['conditions'].unique())
disease_type = list(dt["disease_type"])
fuzzy_matches = {}
for condition in con:
    best_match = process.extractOne(condition, disease_type)
    if best_match[1] > 80:  # Example threshold for similarity
        fuzzy_matches[condition] = best_match[0]

In [4]:
#disease map
dm = pd.DataFrame({'conditions': con})

# Create a dictionary with zero-filled columns for all diseases
disease_columns = {disease: [0] * len(dm) for disease in disease_type}

# Concatenate the columns to the original DataFrame
dm = pd.concat([dm, pd.DataFrame(disease_columns)], axis=1)

In [5]:
no_ones_mask = (dm.iloc[:, 1:] == 0).all(axis=1)
conditions_without_ones = dm.loc[no_ones_mask, 'conditions']

# Step 2: Split the conditions by space and parentheses, and clean the words
all_words = []
for condition in conditions_without_ones:
    # Split by space and parentheses, and clean up empty strings
    words = re.findall(r'\b\w+\b', condition.lower())  # This regex finds words
    all_words.extend(words)

# Step 3: Count the occurrences of each word
word_counts = Counter(all_words)

# Step 4: Retrieve the top 20 words
top_20_words = word_counts.most_common(50)

# Print the top 20 words
print("Top 20 words appearing in conditions without any 1s:")
for word, count in top_20_words:
    print(f"{word}: {count}")

Top 20 words appearing in conditions without any 1s:
stage: 285
cell: 275
lymphoma: 259
cancer: 234
leukemia: 206
acute: 157
adult: 151
carcinoma: 142
recurrent: 139
non: 87
chronic: 87
childhood: 85
of: 74
syndrome: 72
small: 69
lymphoblastic: 69
ii: 69
iv: 64
disease: 63
with: 63
and: 60
myeloid: 59
hodgkin: 58
lung: 54
t: 52
iii: 51
breast: 46
the: 45
metastatic: 43
large: 43
refractory: 42
b: 42
grade: 42
disorder: 40
diffuse: 37
squamous: 35
advanced: 34
follicular: 34
i: 34
tumor: 33
contiguous: 32
neoplasms: 31
adenocarcinoma: 31
s: 31
type: 31
renal: 30
lymphocytic: 30
1: 30
primary: 30
myelodysplastic: 28


In [6]:
non_ones_mask = (dm.iloc[:, 1:] == 0).all(axis=1)

# Filter the DataFrame to include only the rows without any '1's
df_no_ones = dm[non_ones_mask]

# Now find conditions that include the word "cancer"
conditions_with_words = df_no_ones[df_no_ones['conditions'].str.contains("cancer", case=False, na=False)]

# Display the resulting conditions
print(conditions_with_words['conditions'])

0                              non-small cell lung cancer
5                                           breast cancer
6                                         prostate cancer
8                                metastatic breast cancer
11                             non small cell lung cancer
                              ...                        
2221                   castrate resistant prostate cancer
2222                castration resistant prostatic cancer
2226    phase 1 portion : non small cell lung cancer(n...
2227       phase 2 portion : small cell lung cancer(sclc)
2269    advanced or metastatic non-small cell lung cancer
Name: conditions, Length: 238, dtype: object


In [7]:
disease_type = "Cancer"
conditions_list = ["breast cancer", "prostate cancer"]
conditions_list = [x.lower() for x in conditions_list]

In [8]:
for idx, condition in enumerate(dm['conditions']):
    if condition.lower() in conditions_list and (dm.iloc[idx, 1:] == 0).all() and (disease_type in dm.columns):
        dm.loc[idx, disease_type] = 1

In [9]:
no_ones_mask = (dm.iloc[:, 1:] == 0).all(axis=1)

# Count the number of rows where the condition is True (all zeros)
rows_with_no_ones = no_ones_mask.sum()

print(f"Number of rows without any 1s: {rows_with_no_ones}")

Number of rows without any 1s: 2282


In [10]:
no_ones_col_mask = (dm.iloc[:, 1:] == 0).all(axis=0)

# Count the number of columns where the condition is True (all zeros)
cols_with_no_ones = no_ones_col_mask.sum()

print(f"Number of columns without any 1s: {cols_with_no_ones}")

Number of columns without any 1s: 151
